In [1]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip
!unzip datasource.zip


--2025-07-22 03:43:39--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4249 (4.1K) [application/zip]
Saving to: ‘datasource.zip’

datasource.zip      100%[===================>]   4.15K  --.-KB/s    in 0s      

2025-07-22 03:43:40 (1.05 GB/s) - ‘datasource.zip’ saved [4249/4249]

Archive:  datasource.zip
  inflating: used_car_prices1.csv    
  inflating: used_car_prices2.csv    
  inflating: used_car_prices3.csv    
  inflating: used_car_prices1.json   
  inflating: used_car_prices

In [2]:
import glob
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime
log_file = "log_file2.txt"
target_file = "transformed_data2.csv"

def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines = True)
    return dataframe

def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=["car_model","year_of_manufacture","price","fuel"])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for car in root:
        car_model = car.find("car_model").text
        year_of_manufacture = car.find("year_of_manufacture").text
        price = float(car.find("price").text)
        fuel = car.find("fuel").text

        dataframe = pd.concat([dataframe,pd.DataFrame([{"car_model": car_model,"year_of_manufacture":year_of_manufacture,"price":price,"fuel":fuel}])],ignore_index =True)
    return dataframe

def extract():
    extracted_data = pd.DataFrame(columns=["car_model","year_of_manufacture","price","fuel"])
    for csvfile in glob.glob("*.csv"):
        if csvfile != target_file:
            extracted_data = pd.concat([extracted_data,pd.DataFrame(extract_from_csv(csvfile))],ignore_index=True)

    for jsonfile in glob.glob("*.json"):
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_json(jsonfile))], ignore_index=True)

    for xmlfile in glob.glob("*.xml"):
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_xml(xmlfile))], ignore_index=True)

    return extracted_data

def transform(data):
    data['price'] = data['price'].round(2)


    return data

def load_data(target_file,transformed_data):
    transformed_data.to_csv(target_file)


def log_progress(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open(log_file,"a") as f:
        f.write(timestamp + ',' + message + '\n')

log_progress("ETL JOB STARTED")
log_progress("EXTRACT PHASE START")
extracted_data = extract()
log_progress("EXTRACT PHASE END")

log_progress("TRANSFORM PHASE START")
transformed_data = transform(extracted_data)
print("TRANSFORMED DATA")
print(transformed_data)

log_progress("TRANSFORM PHASE ENDED")

log_progress("LOAD PHASE START")
load_data(target_file,transformed_data)
log_progress("LOAD PHASE END")

log_progress("ETL JOB END")

TRANSFORMED DATA
        car_model year_of_manufacture     price    fuel
0            ritz                2014   5000.00  Petrol
1             sx4                2013   7089.55  Diesel
2            ciaz                2017  10820.90  Petrol
3         wagon r                2011   4253.73  Petrol
4           swift                2014   6865.67  Diesel
..            ...                 ...       ...     ...
85        etios g                2015   5895.52  Petrol
86  corolla altis                2013   8208.96  Petrol
87        corolla                2004   2238.81  Petrol
88  corolla altis                2010   7835.82  Petrol
89       fortuner                2012  21641.79  Diesel

[90 rows x 4 columns]


/tmp/ipython-input-2-2061207675.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  extracted_data = pd.concat([extracted_data,pd.DataFrame(extract_from_csv(csvfile))],ignore_index=True)
/tmp/ipython-input-2-2061207675.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframe = pd.concat([dataframe,pd.DataFrame([{"car_model": car_model,"year_of_manufacture":year_of_manufacture,"price":price,"fuel":fuel}])],ignore_index =True)
/tmp/ipython-input-2-2061207675.py:26: FutureWarning: The behavi